# Generalizing Chunks
#### John R. Starr; jrs294@pitt.edu
Now that we have all of the data POS-tagged and chunked, it's time to generalize the chunks into categories: SOV, SVO, and an "extraneous" column EX (which will probably be filled by mis-parsed/mis-chunked data). At the most basic level, I intend on examining the number of noun phrases before the verb. If there are two, then we'll make it one. 

NOTE: I understand that the chunked data I have is by no means perfect; this is one of the limitations of my project.

All right, let's load in the usual stuff:

In [1]:
import nltk
import numpy as np
import pandas as pd

In [2]:
full_df = pd.read_pickle('tagged_chunked_df.pkl')

In [3]:
full_df.head()

,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks
ID,,,,,,,,,,,,
1,raspy breathing,صداي خر خر,"[raspy, breathing]","[صداي, خر, خر]",2,3,"{raspy, breathing}","{خر, صداي}","[(صداي, NUM), (خر, Ne), (خر, N)]",[صداي خر خر NP],"[(raspy, NN), (breathing, NN)]","[[(raspy, NN), (breathing, NN)]]"
2,dad,پدر,[dad],[پدر],1,1,{dad},{پدر},"[(پدر, N)]",[پدر NP],"[(dad, NN)]","[[(dad, NN)]]"
3,maybe its the wind,شايد صداي باد باشه,"[maybe, its, the, wind]","[شايد, صداي, باد, باشه]",4,4,"{its, the, wind, maybe}","{باشه, باد, صداي, شايد}","[(شايد, Ne), (صداي, AJ), (باد, V), (باشه, V)]",[شايد صداي NP] [باد VP] [باشه VP],"[(maybe, RB), (its, PRP$), (the, DT), (wind, NN)]","[(maybe, RB), (its, PRP$), [(the, DT), (wind, ..."
4,no,نه,[no],[نه],1,1,{no},{نه},"[(نه, ADV)]",نه,"[(no, DT)]","[[(no, DT)]]"
5,stop please stop,دست نگه داريد خواهش ميکنم دست نگه داريد,"[stop, please, stop]","[دست, نگه, داريد, خواهش, ميکنم, دست, نگه, داريد]",3,8,"{stop, please}","{خواهش, ميکنم, دست, داريد, نگه}","[(دست, N), (نگه, N), (داريد, V), (خواهش, Ne), ...",[دست NP] [نگه داريد VP] [خواهش ميکنم دست NP] [...,"[(stop, JJ), (please, NN), (stop, VB)]","[[(stop, JJ), (please, NN)], [[('stop', 'VB')]]]"


Rather than working on the full DF, let's create a smaller one that we can run functions on faster...:

In [4]:
small_df = full_df.iloc[:100]

Let's see what type of data we're working with. Because we're looking for the abnormalities in Persian word order, we only need to focus on that data. So, what's it all look like?

In [5]:
# Gathering some information on the Persian data:
for item in small_df.Far_Chunks.iloc[:5]:
    print(item)
    print(item[0])
    print()

[صداي خر خر NP]
[

[پدر NP]
[

[شايد صداي NP] [باد VP] [باشه VP]
[

نه
ن

[دست NP] [نگه داريد VP] [خواهش ميکنم دست NP] [نگه داريد VP]
[



Ah.. it seems that our data is in string format...! This makes things a little more annoying to navigate. If only the data were in tuple form! Oh well...  Let's try using RegEx to search through this and get the word order that we want.

NOTE: Persian reads from right-to-left; however, the chunker flips the sentence backward so that it can be read "left-to-right" and can be more easily compared with languages like English. If this is confusing now, I'll explain more in my presentation.

In [6]:
# Importing...
import re

Let's try a test regex that looks for SVO word-order:

In [7]:
# SVO
SVO_test = []
for x in small_df.Far_Chunks:
    if re.findall('(NP)[^NP]*VP[^VP]*(NP)+[^VP]*', x):
        # The goal of the regex is as follows:
            # One NP, then any characters besides another NP, then a VP, then an NP (presumably the object)
        SVO_test.append(x)
len(SVO_test)

33

What kind of sentences are we looking at right now?

In [8]:
SVO_test[10:20]

['[مارک اول اينکه من NP] [ميخوام شما NP] [پسرهارو NP] [با PP] [خودم NP] [ببرم VP] و [ما NP] [گله NP] [را POSTP] [جمع NP] [خواهيم کرد VP] [بعد ADVP] [من NP] [به PP] [شهر ميرم NP]',
 '[من NP] [به PP] [هلندر NP] [ميگم VP] که [اينو NP] [درستش ADJP] [کنه VP]',
 '[من NP] [بهش NP] [ميگم VP] [پول يک اسطبل NP] [جديدو بده VP]',
 '[شايد ما NP] [بايد VP] [بهش PP] [تير NP] [اندازي VP] کنيم [همونجور NP] که [ويل ميگه NP]',
 '[تو NP] [بايد VP] [ميگذاشتي من NP] [علوفه هارو NP] [نگه دارم VP]',
 '[يه NP] [روزي VP] که [مثل PP] [من NP] [شدي VP] [تو NP] [اونوقت ميفهمي NP]',
 '[خوب ADJP] [يک نگاه NP] [به PP] [اينا NP] [ميکني VP] [همه چيزهايي NP] که [داري PP] [هيچ مصرفي NP] [نداره VP]',
 '[هيچ چيزي NP] [عوض نميشه VP] اگر [من NP] [زنده ADJP] [بمونم VP]',
 '[ميخوام NP] که [پول هارو NP] [بذارين زمين NP] [بريد VP] [عقب بهشون ADJP] [بگو VP] [برن NP] [عقب ADVP] [يا اين مرد ميميره NP]',
 '[الان ADVP] [ميدونم VP] که [چارلي NP] [بهت PP] [چي گفته NP] [براي VP] اينکه [ما NP] چيزي [تو NP] [اين گروه NP] [نداريم VP] [بجز P

Unfortunately, my chunker isn't the best. There are some clear errors with its labeling. For example, the first sentence up above has the [ميخوام شما] labeled as an "NP", even though "میخواهم" is a verb. It also doesn't take into account colloquial spellings of certain verbs [برم]...! The reason behind this is that Persian varies greatly between its spoken and written forms.

All things considered, though, this looks pretty good! The patterns I want are all in the data as best that the chunker allows...!

Now, let's check out SOV sentences. I predict that there will be more of these:

In [9]:
# SOV
SOV_test = []
for x in small_df.Far_Chunks[:100]:
    if re.findall('(NP)[^VP]*(NP)+(VP)*[^NP]', x):
        SOV_test.append(x)
len(SOV_test)

38

In [10]:
SOV_test[10:20]

['[تو NP] [ميخواي NP] [به PP] [مارشال NP] [بگي VP] که [اون آدما چيکار کردن NP]',
 '[مارک اول اينکه من NP] [ميخوام شما NP] [پسرهارو NP] [با PP] [خودم NP] [ببرم VP] و [ما NP] [گله NP] [را POSTP] [جمع NP] [خواهيم کرد VP] [بعد ADVP] [من NP] [به PP] [شهر ميرم NP]',
 '[من NP] [بهش NP] [ميگم VP] [پول يک اسطبل NP] [جديدو بده VP]',
 '[شايد ما NP] [بايد VP] [بهش PP] [تير NP] [اندازي VP] کنيم [همونجور NP] که [ويل ميگه NP]',
 '[دور NP] [تا PP] [دورش NP] [از PP] [آهنه يک NP] [چيزايي بالاش NP] [دوتا شاتگان زن و يک تيربار NP]',
 '[تو NP] [بايد VP] [ميگذاشتي من NP] [علوفه هارو NP] [نگه دارم VP]',
 '[يه NP] [روزي VP] که [مثل PP] [من NP] [شدي VP] [تو NP] [اونوقت ميفهمي NP]',
 '[اون تويي NP] [خانم کوچولو NP]',
 '[من NP] [بايد VP] [بگم VP] که [احتمالا ADVP] [بي PP] [ارزشتر NP] [از PP] [اونيه NP] که [من NP] [بخوام VP] [چيزي ازش بدزدم VP]',
 '[من NP] [تورو NP] [نميخوام بکشم VP]']

This looks a _lot_ better than the SVO output, which isn't unexpected -- since Persian tends towards SOV word order, it makes sense that the data here is much cleaner. Now let's build a function that will do all this automatically...! We're going to look for SVO, SOV, and "Other" word order, with "Other" covering the sentence fragments or other strange speech patterns that appear in everyday language.

Building a function:

In [11]:
def gen_word_order(text):
    if re.findall('(NP)[^NP]*VP[^VP]*(NP)+[^VP]*', text):
        return 'SVO'
    elif re.findall('(NP)[^VP]*(NP)+(VP)*[^NP]', text):
        return 'SOV'
    else:
        return 'Other'

How's it apply to our small_df?

In [12]:
small_df['Word_Order'] = small_df['Far_Chunks'].apply(gen_word_order)

C:\Users\16starjo\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
small_df.tail(10)

,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
ID,,,,,,,,,,,,,
91,by mr ben wade himself,بوسيله خود بن ويد,"[by, mr, ben, wade, himself]","[بوسيله, خود, بن, ويد]",5,4,"{ben, by, himself, mr, wade}","{بن, ويد, خود, بوسيله}","[(بوسيله, Ne), (خود, PRO), (بن, Ne), (ويد, N)]",[بوسيله خود بن ويد NP],"[(by, IN), (mr, JJ), (ben, NN), (wade, VBD), (...","[[[('by', 'IN')], [('mr', 'JJ'), ('ben', 'NN')...",Other
92,how did you know it was wade,تو از کجا فهميدي که اون ويد بود,"[how, did, you, know, it, was, wade]","[تو, از, کجا, فهميدي, که, اون, ويد, بود]",7,8,"{know, you, it, did, how, was, wade}","{فهميدي, کجا, بود, از, که, تو, ويد, اون}","[(تو, PRO), (از, P), (کجا, ADV), (فهميدي, V), ...",[تو NP] [از PP] [کجا NP] [فهميدي VP] که [اون و...,"[(how, WRB), (did, VBD), (you, PRP), (know, VB...","[(how, WRB), [[('did', 'VBD')]], (you, PRP), [...",SVO
93,its been him the last 21 times marshal,اون خودش بود توي 21 بار قبلي مارشال,"[its, been, him, the, last, 21, times, marshal]","[اون, خودش, بود, توي, 21, بار, قبلي, مارشال]",8,8,"{its, marshal, the, times, been, him, last, 21}","{مارشال, قبلي, توي, بود, خودش, بار, اون, 21}","[(اون, PRO), (خودش, PRO), (بود, V), (توي, N), ...",[اون NP] [خودش NP] [بود VP] [توي NP] [21 بار A...,"[(its, PRP$), (been, VBN), (him, PRP), (the, D...","[(its, PRP$), [[('been', 'VBN')]], (him, PRP),...",SVO
94,i saw a mexican sharpshooter and an apache,من يک تير انداز ماهر مکزيکي و يک آپاچي باهاش ديدم,"[i, saw, a, mexican, sharpshooter, and, an, ap...","[من, يک, تير, انداز, ماهر, مکزيکي, و, يک, آپاچ...",8,11,"{apache, a, sharpshooter, an, i, saw, and, mex...","{يک, من, باهاش, ديدم, تير, مکزيکي, آپاچي, اندا...","[(من, PRO), (يک, NUM), (تير, Ne), (انداز, AJe)...",[من NP] [يک تير انداز ماهر NP] [مکزيکي VP] و [...,"[(i, NN), (saw, VBD), (a, DT), (mexican, JJ), ...","[[(i, NN)], [[('saw', 'VBD')], [('a', 'DT'), (...",SVO
95,did you see the hand of god whats that,تو دست خدا را ديدي اون چيه ديگه,"[did, you, see, the, hand, of, god, whats, that]","[تو, دست, خدا, را, ديدي, اون, چيه, ديگه]",9,8,"{you, see, that, of, the, did, whats, hand, god}","{چيه, دست, را, ديدي, خدا, ديگه, تو, اون}","[(تو, PRO), (دست, Ne), (خدا, N), (را, POSTP), ...",[تو NP] [دست خدا NP] [را POSTP] [ديدي PP] [اون...,"[(did, VBD), (you, PRP), (see, VB), (the, DT),...","[[[('did', 'VBD')]], (you, PRP), [[('see', 'VB...",SOV
96,his pistol why the hell didnt you do something,هفت تيرش چرا تو هيچ کاري نکردي,"[his, pistol, why, the, hell, didnt, you, do, ...","[هفت, تيرش, چرا, تو, هيچ, کاري, نکردي]",9,7,"{pistol, you, didnt, hell, the, something, why...","{تيرش, نکردي, هفت, چرا, کاري, تو, هيچ}","[(هفت, NUM), (تيرش, N), (چرا, ADV), (تو, PRO),...",[هفت تيرش NP] [چرا ADVP] [تو NP] [هيچ کاري NP]...,"[(his, PRP$), (pistol, NN), (why, WRB), (the, ...","[(his, PRP$), [(pistol, NN)], (why, WRB), [(th...",SVO
97,they had a lot of weapons mr and they were sh...,اونا کلي اسلحه داشتن آقا و اونا گلوله شليک ميکردن,"[they, had, a, lot, of, weapons, mr, and, they...","[اونا, کلي, اسلحه, داشتن, آقا, و, اونا, گلوله,...",12,10,"{a, shooting, and, of, mr, they, had, weapons,...","{اسلحه, آقا, شليک, اونا, ميکردن, داشتن, گلوله,...","[(اونا, PRO), (کلي, CONJ), (اسلحه, N), (داشتن,...",[اونا NP] کلي [اسلحه داشتن آقا و اونا گلوله شل...,"[(they, PRP), (had, VBD), (a, DT), (lot, NN), ...","[(they, PRP), [[('had', 'VBD')], [('a', 'DT'),...",SOV
98,lets go were wasting time,بريم داريم وقت تلف ميکنيم,"[lets, go, were, wasting, time]","[بريم, داريم, وقت, تلف, ميکنيم]",5,5,"{wasting, go, were, time, lets}","{وقت, تلف, بريم, ميکنيم, داريم}","[(بريم, N), (داريم, V), (وقت, Ne), (تلف, Ne), ...",[بريم NP] [داريم VP] [وقت تلف ميکنيم NP],"[(lets, NNS), (go, VBP), (were, VBD), (wasting...","[(lets, NNS), [[('go', 'VBP')]], [[('were', 'V...",SVO
99,where are you from anyway,درهرصورت تو کجايي هستي,"[where, are, you, from, anyway]","[درهرصورت, تو, کجايي, هستي]",5,4,"{where, anyway, you, are, from}","{تو, درهرصورت, کجايي, هستي}","[(درهرصورت, CONJ), (ت

What are the value counts of everything we've seen thus far?

In [14]:
small_df['Word_Order'].value_counts()

Other    50
SVO      33
SOV      17
Name: Word_Order, dtype: int64

Hmmm... these numbers seem a bit funky...! We got 38 SOV values when we searched through the small_df, but only 17 when we applied it to our DF -- however, our SVO value is the same! I wonder if the regexps are stepping on each other's toes? Let's reverse the rule and see what happens:

In [15]:
def gen_word_order2(text):
    if re.findall('(NP)[^VP]*(NP)+(VP)*[^NP]', text):
        return 'SOV'
    elif re.findall('(NP)[^NP]*VP[^VP]*(NP)+[^VP]*', text):
        return 'SVO'
    else:
        return 'Other'

In [16]:
small_df['Word_Order2'] = small_df['Far_Chunks'].apply(gen_word_order2)

C:\Users\16starjo\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
small_df['Word_Order2'].value_counts()

Other    50
SOV      38
SVO      12
Name: Word_Order2, dtype: int64

Yes, the regular expressions most definitely _are_ treading on each other...! This time, all the SOV values were filled, and whatever crossover remained went to SVO! That needs to be fixed. Also, it is interesting to note that the regular expressions only apply to 50% of the data; we'll see how this scales as we work up to the full_df.

How much crossover do we have?

In [18]:
crossover = []
for x in SOV_test:
    if x in SVO_test:
        crossover.append(x)
len(crossover)

21

Not great...! We need to fix the regex. Still, let's take a look at the crossover:

In [19]:
crossover

['[اين آخرين علوفه NP] [اي NP] [بود VP] که [ما NP] [داشتيم VP]',
 '[تو NP] [به PP] [من NP] [گفتي VP] [قرضمونو NP] [به PP] [هلندر NP] [پرداخت کرديم ما NP] [پرداخت کرديم NP]',
 '[خب ADVP] [پس ADVP] [تو NP] [فکر ميکني ما NP] [چطوري VP] [علوفه خريديم NP] [سه ماه NP] [آب خرج دارو مارک NP]',
 '[من NP] [فقط ADVP] [يک PP] [انتخاب بين NP] [خانوادم VP] و [پرداخت قرض NP] [اونا NP] [داشتم VP]',
 '[ما NP] [قرار NP] [بود VP] [تصميماتو NP] [با PP] [هم NP] [بگيريم VP]',
 '[مشکل NP] [اينجاست VP] که [دکترها NP] [در PP] [درمانگاه کار زيادي NP] [براي VP] [پاي PP] [من NP] [نکردن مستمري NP] [هم ADVP] [الان ADVP] [به PP] [پوند NP] [پرداخت ميشه NP]',
 '[تو NP] [ميخواي NP] [به PP] [مارشال NP] [بگي VP] که [اون آدما چيکار کردن NP]',
 '[مارک اول اينکه من NP] [ميخوام شما NP] [پسرهارو NP] [با PP] [خودم NP] [ببرم VP] و [ما NP] [گله NP] [را POSTP] [جمع NP] [خواهيم کرد VP] [بعد ADVP] [من NP] [به PP] [شهر ميرم NP]',
 '[من NP] [بهش NP] [ميگم VP] [پول يک اسطبل NP] [جديدو بده VP]',
 '[شايد ما NP] [بايد VP] [بهش PP] [تير N

So, many of these files have complex structures that include multiple NPs and VPs. Unforuntately, the strings do not have punctuation, so I can't split the sentences up on that. Admittedly, this is kind of giving me a headache.

But still, time to fix things!! Thanks to some help from Professor Han, I have tried something else that works better. Here are some sample strings to test on:

In [20]:
s1 = '[;asldkfjlsdk NP] [sakdjfijk VP]'
s2 = '[alskdjflksdj NP] [:DLkjasdlkfj VP] [alskdfj NP]'
s3 = '[alskdjflksdj NP] [:DLkjasdlkfj NP] [alskdfj VP]'
s4 = '[alskdjflksdj NP] [:DLkjasdlkfj NP] [alskdfj VP] [alskdjf NP]'
s5 = '[alskdjflksdj NP] [:DLkjasdlkfj AP] [alskdfj VP] [askdfj NP]'
s6 = '[alskdjflksdj NP] [:DLkjasdlkfj AP] [alskdfj NP] [askdfj VP]'

First we'll build structures that DO represent NPs/VPs:

In [21]:
np = r'\[[^\]]+ NP\]'
vp = r'\[[^\]]+ VP\]'

And then structures that DO NOT represent NPs/VPs:

In [22]:
np_not = r'\[[^\]]+ [^N]P\]'
vp_not = r'\[[^\]]+ [^V]P\]'

Now to build the SVO and SOV word order patterns:

In [23]:
# Matching for SVO
svo_pat = re.compile( r""+ np +" ("+ np_not +" )*"+ vp +" "+ np)

In [24]:
# Testing match
svo_pat.match(s5)

<re.Match object; span=(0, 60), match='[alskdjflksdj NP] [:DLkjasdlkfj AP] [alskdfj VP] >

In [25]:
# Matching for SOV
sov_pat = re.compile( r""+ np +" ("+ vp_not +" )*"+ np + " " + vp)

In [26]:
# Testing match
sov_pat.match(s6)

<re.Match object; span=(0, 60), match='[alskdjflksdj NP] [:DLkjasdlkfj AP] [alskdfj NP] >

Well, let's see what changes when we use these two functions instead:

In [27]:
def gen_word_order3(text):
    if svo_pat.match(text):
        return 'SVO'
    elif sov_pat.match(text):
        return 'SOV'
    else:
        return 'Other'

In [28]:
small_df['Word_Order3'] = small_df['Far_Chunks'].apply(gen_word_order3)

C:\Users\16starjo\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
small_df['Word_Order3'].value_counts()

Other    75
SOV      21
SVO       4
Name: Word_Order3, dtype: int64

And if I flip the function? Last time, this was a good way of checking whether or not the ordering of the if/elif/else statements was affecting the numbers:

In [30]:
def gen_word_order4(text):
    if sov_pat.match(text):
        return 'SOV'
    elif svo_pat.match(text):
        return 'SVO'
    else:
        return 'Other'

In [31]:
small_df['Word_Order4'] = small_df['Far_Chunks'].apply(gen_word_order4)

C:\Users\16starjo\Documents\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
small_df['Word_Order4'].value_counts()

Other    75
SOV      21
SVO       4
Name: Word_Order4, dtype: int64

Well, at least this one doesn't have any cross-over! And, as expected, there are more SOV structures.

Unfortunately, this function ignores 75% of the small_df. That's a _lot_ of data to be losing. We'll need to see how this scales up when applied to the full DF.

A quick refresher on what our full DF looks like:

In [33]:
full_df.head()

,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks
ID,,,,,,,,,,,,
1,raspy breathing,صداي خر خر,"[raspy, breathing]","[صداي, خر, خر]",2,3,"{raspy, breathing}","{خر, صداي}","[(صداي, NUM), (خر, Ne), (خر, N)]",[صداي خر خر NP],"[(raspy, NN), (breathing, NN)]","[[(raspy, NN), (breathing, NN)]]"
2,dad,پدر,[dad],[پدر],1,1,{dad},{پدر},"[(پدر, N)]",[پدر NP],"[(dad, NN)]","[[(dad, NN)]]"
3,maybe its the wind,شايد صداي باد باشه,"[maybe, its, the, wind]","[شايد, صداي, باد, باشه]",4,4,"{its, the, wind, maybe}","{باشه, باد, صداي, شايد}","[(شايد, Ne), (صداي, AJ), (باد, V), (باشه, V)]",[شايد صداي NP] [باد VP] [باشه VP],"[(maybe, RB), (its, PRP$), (the, DT), (wind, NN)]","[(maybe, RB), (its, PRP$), [(the, DT), (wind, ..."
4,no,نه,[no],[نه],1,1,{no},{نه},"[(نه, ADV)]",نه,"[(no, DT)]","[[(no, DT)]]"
5,stop please stop,دست نگه داريد خواهش ميکنم دست نگه داريد,"[stop, please, stop]","[دست, نگه, داريد, خواهش, ميکنم, دست, نگه, داريد]",3,8,"{stop, please}","{خواهش, ميکنم, دست, داريد, نگه}","[(دست, N), (نگه, N), (داريد, V), (خواهش, Ne), ...",[دست NP] [نگه داريد VP] [خواهش ميکنم دست NP] [...,"[(stop, JJ), (please, NN), (stop, VB)]","[[(stop, JJ), (please, NN)], [[('stop', 'VB')]]]"


And applying our word_order function to it:

In [34]:
def gen_word_order_final(text):
    if sov_pat.match(text):
        return 'SOV'
    elif svo_pat.match(text):
        return 'SVO'
    else:
        return 'Other'

In [35]:
full_df['Word_Order'] = full_df['Far_Chunks'].apply(gen_word_order_final)

In [36]:
print(full_df['Word_Order'].value_counts())
print()
print(len(full_df))

Other    535371
SOV       52756
SVO       23959
Name: Word_Order, dtype: int64

612086


Well, this is _not_ ideal whatsoever. Over 87% of the data is labeled as "other". There are a few explanations for this:
- Chunker mischunked the data, therefore giving incorrect structures
- Regex are too restrictive in their selection (need editing)
- Various factors do not take into account the pro-drop nature of Persian, so they don't recognize OV structures to be SOV (create a separate OV category?)

Due to time constraints, I don't think I'll be able to fix this right now, but hopefully by the time I present on Tuesday I'll have a better grasp on my data...! For now, let's separate the structures that are properly labeled, pickle them out, then perform data analysis in another notebook.

First, I'll create a list of Boolean values that will distinguish between the "Other" and ordered cases.

In [37]:
booleans = []
for order in full_df['Word_Order']:
    if order == 'Other':
        booleans.append(False)
    else:
        booleans.append(True)

In [38]:
# Checking the length
len(booleans)

612086

Now we'll turn this list into a Series so that way we can use it to organize the DF:

In [39]:
is_ordered = pd.Series(booleans)

Wait! The index for full_df starts at 1, not 0. Let's reset that:

In [40]:
full_df.reset_index(inplace = True)

Now to create the "ordered only" DF:

In [41]:
ordered_only_df = full_df[is_ordered]

In [42]:
ordered_only_df.head()

,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
4,5,stop please stop,دست نگه داريد خواهش ميکنم دست نگه داريد,"[stop, please, stop]","[دست, نگه, داريد, خواهش, ميکنم, دست, نگه, داريد]",3,8,"{stop, please}","{خواهش, ميکنم, دست, داريد, نگه}","[(دست, N), (نگه, N), (داريد, V), (خواهش, Ne), ...",[دست NP] [نگه داريد VP] [خواهش ميکنم دست NP] [...,"[(stop, JJ), (please, NN), (stop, VB)]","[[(stop, JJ), (please, NN)], [[('stop', 'VB')]]]",SVO
8,9,god damn it put that down,لعنت به تو اونو بذار زمين,"[god, damn, it, put, that, down]","[لعنت, به, تو, اونو, بذار, زمين]",6,6,"{god, that, it, put, down, damn}","{لعنت, به, زمين, اونو, بذار, تو}","[(لعنت, N), (به, P), (تو, PRO), (اونو, PRO), (...",[لعنت NP] [به PP] [تو NP] [اونو NP] [بذار VP] ...,"[(god, NN), (damn, VBZ), (it, PRP), (put, VBD)...","[[(god, NN)], [[('damn', 'VBZ')]], (it, PRP), ...",SOV
10,11,its the last feed weve got,اين آخرين علوفه اي بود که ما داشتيم,"[its, the, last, feed, weve, got]","[اين, آخرين, علوفه, اي, بود, که, ما, داشتيم]",6,8,"{its, the, feed, last, weve, got}","{ما, اي, علوفه, داشتيم, بود, آخرين, که, اين}","[(اين, Ne), (آخرين, NUM), (علوفه, N), (اي, N),...",[اين آخرين علوفه NP] [اي NP] [بود VP] که [ما N...,"[(its, PRP$), (the, DT), (last, JJ), (feed, NN...","[(its, PRP$), [(the, DT), (last, JJ), (feed, N...",SOV
14,15,you lied to me dan,تو به من دروغ گفتي دن,"[you, lied, to, me, dan]","[تو, به, من, دروغ, گفتي, دن]",5,6,"{you, lied, me, to, dan}","{دن, من, گفتي, به, دروغ, تو}","[(تو, PRO), (به, P), (من, PRO), (دروغ, AJ), (گ...",[تو NP] [به PP] [من NP] [دروغ NP] [گفتي VP] دن,"[(you, PRP), (lied, VBD), (to, TO), (me, PRP),...","[(you, PRP), [[('lied', 'VBD')]], (to, TO), (m...",SOV
15,16,you told me we made payments to hollander we did,تو به من گفتي قرضمونو به هلندر پرداخت کرديم ما...,"[you, told, me, we, made, payments, to, hollan...","[تو, به, من, گفتي, قرضمونو, به, هلندر, پرداخت,...",10,12,"{we, you, told, hollander, payments, did, me, ...","{ما, پرداخت, کرديم, من, گفتي, به, هلندر, تو, ق...","[(تو, PRO), (به, P), (من, PRO), (گفتي, V), (قر...",[تو NP] [به PP] [من NP] [گفتي VP] [قرضمونو NP]...,"[(you, PRP), (told, VBD), (me, PRP), (we, PRP)...","[(you, PRP), [[('told', 'VBD')]], (me, PRP), (...",SOV


Pickle time!

In [43]:
ordered_only_df.to_pickle('ordered_only_df.pkl')

And that's it for now...! Time for some data analysis!